#CH 1

# 자연어 처리 모델
- 자연어 처리 모델은 자연어를 입력받아서 해당 입력이 특정 범주일 확률을 반환하는 확률 함수
- 그 중에서도 딥러닝을 이용한 자연어 처리 모델
  - BERT (Bidirectional Encoder Representations from Transformers)
  - GPT (Generative Pre-trained Transformer)

# Transfer leaning (전이 학습)
- 특정 태스크를 학습한 모델을 다른 태스크 수행에 재사용하는 기법
- 기존보다 모델의 학습 속도가 빨라지고 새로운 태스크를 더 잘 수행하는 경향이 있다
  - Upstream task : 기존의 태스크, 대규모 말뭉치의 문맥을 이해하는 과제
    - Pretrain : Upstream task를 학습하는 과정
  - Downstream task : 새로운 태스크, 풀고자 하는 자연어 처리의 구체적인 문제들
  

## Upsteram task
1. 다음 단어 맞히기
  - GPT 계열
  - Language Model
2. 빈칸 채우기
  - BERT 계열
  - Masked Language Model

## Downstream task
- Classification을 수행
- Fine tunning : Downstream task의 학습 방식 중 하나
  - Pretrain을 마친 모델을 Downstream task에 맞게 업데이트하는 기법
  - 이 외에도 Prompt tunning, In context tunning 존재
1. 문서 분류
2. 자연어 추론
3. 개체명 인식
4. 질의응답
5. 문장 생성

#CH 2

# Tokenization
- 문장을 토큰 시퀀스로 나누는 과정
- Tokenizer : 토큰화를 수행하는 프로그램
1. 단어 단위 토큰화 : 어휘 집합의 크기가 매우 커짐
2. 문자 단위 토큰화 : 의미 있는 단어가 되기 어려움
3. 서브워드 단위 토큰화 : 단어와 문자 단위 토큰화의 중간에 있는 형태
  - 바이트 페어 인코딩

### Byte Pair Encoding (BPE,바이트 페어 인코딩)
- 사전 크기 증가를 억제하면서도 정보를 효율적으로 압축할 수 있음
- 말뭉치에서 자주 나타나는 문자열(서브워드)을 토큰으로 분석함
1. 어휘 집합 구축
  - 자주 등장하는 문자열을 병합하고 이를 어휘 집합에 추가. 이를 원하는 어휘 집합 크기가 될 때가지 반복
  - Pre-tokenize : 말뭉치의 모든 문장을 공백으로 나눔
2. 토큰화 
  - 토큰화 대상 문장의 각 어절에서 어휘 집합에 있는 서브워드가 포함되었을 때 해당 서브워드를 어절에서 분리

### Wordpiece
- 말뭉치에서 자주 등장한 문자열을 토큰으로 인식한다는 점에서 BPE와 본질적으로 유사함
- 하지만 wordpiece는 단순히 빈도를 기준으로 병합하는 것이 아니라, 병합했을 때 말뭉치의 '우도'를 가장 높이는 쌍을 병합
- BPE와 달리 merge.txt 생성하지 않음, vocab.txt만 사용

# Tokenizer 생성

In [ ]:
!pip install ratsnlp

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
from Korpora import Korpora
nsmc = Korpora.load('nsmc', force_download=True)

In [ ]:
# NSMC 전치
import os
def write_lines(path, lines):
  with open(path, 'w', encoding='utf-8') as f:
    for line in lines:
      f.write(f'{line}\n')

write_lines('/root/train.txt', nsmc.train.get_all_texts())
write_lines('/root/test.txt', nsmc.test.get_all_texts())

In [ ]:
# 디렉토리 생성
import os
os.makedirs('/gdrive/My Drive/nlpbook/bbpe', exist_ok=True)

In [ ]:
# 바이트 수준 BPE 어휘 집합 구축
# GPT 토크나이저 - BPE
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=['/root/train.txt','/root/test.txt'], # 학습 말뭉치를 리스트 형태로 넣기
    vocab_size = 10000, # 어휘 집합 ㅡ기 조절
    special_tokens=['[PAD]'] # 특수 토큰 추가
)
bytebpe_tokenizer.save_model('/gdrive/My Drive/nlpbook/bbpe')

['/gdrive/My Drive/nlpbook/bbpe/vocab.json',
 '/gdrive/My Drive/nlpbook/bbpe/merges.txt']

In [ ]:
# BERT 토크나이저 - Wordpiece
import os
os.makedirs('/gdrive/My Drive/nlpbook/wordpiece',exist_ok=True)

from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=['/root/train.txt', '/root/test.txt'],
    vocab_size=10000
)

wordpiece_tokenizer.save_model('/gdrive/My Drive/nlpbook/wordpiece')

['/gdrive/My Drive/nlpbook/wordpiece/vocab.txt']

# 토큰화하기

In [ ]:
# GPT 토크나이저 선언
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained('/gdrive/My Drive/nlpbook/bbpe')
tokenizer_gpt.pad_token = '[PAD]'

file /gdrive/My Drive/nlpbook/bbpe/config.json not found


In [ ]:
# GPT 토크나이저로 토큰화하기
sentences = [
    '아 더빙.. 진짜 짜증나네요 목소리',
    '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
    '별루 였다..'
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

In [ ]:
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

In [ ]:
# GPT 모델 입력 만들기
batch_inputs = tokenizer_gpt(
    sentences,
    padding='max_length', # 문장의 최대 길이에 맞춰 패딩
    max_length = 12, # 문장의 토큰 기준 최대 길이
    truncation = True, # 문장 잘림 허용 옵션
)

In [ ]:
batch_inputs['input_ids'] # 토큰 인덱스 시퀀스

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
batch_inputs['attention_mask'] # 일반 토큰(1), 패딩 토큰(0)

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
# BERT 토크나이저 선언
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    '/gdrive/My Drive/nlpbook/wordpiece',
    do_lower_case = False
)

file /gdrive/My Drive/nlpbook/wordpiece/config.json not found


In [ ]:
# BERT 토크나이저로 토큰화하기
sentences = [
    '아 더빙.. 진짜 짜증나네요 목소리',
    '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
    '별루 였다..'
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

In [ ]:
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

In [ ]:
# BERT 모델 입력 만들기
batch_inputs = tokenizer_bert(
    sentences,
    padding='max_length',
    max_length = 12,
    truncation = True
)

In [ ]:
batch_inputs['input_ids'] # 토큰 인덱스 시퀀스

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1097, 16, 3],
 [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

- 모든 문장 앞에 2, 끝에 3이 붙음
  - 각각 [CLS], [SEP]라는 토큰에 대응하는 인덱스

In [ ]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]